# Transformation

In [0]:

query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY cc.customer_id) AS customer_key,
    cc.customer_id,
    cc.customer_number,
    cc.first_name,
    cc.last_name,
    ecl.country,
    cc.marital_status,
    CASE
        WHEN cc.gender <> 'n/a' THEN cc.gender
        ELSE COALESCE(ec.gender, 'n/a')
    END AS gender,
    ec.birth_date AS birthdate,
    cc.created_date AS create_date
FROM silver.crm_customers cc
LEFT JOIN silver.erp_customers ec ON cc.customer_number = ec.customer_number
LEFT JOIN silver.erp_customer_location ecl ON cc.customer_number = ecl.customer_number
"""
df = spark.sql(query)

In [0]:
df.limit(5).display()

# Writing Gold Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")

### Sanity checks

In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers LIMIT 10